In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import numpy as np

#data path 수정
os.chdir("C:\\Users\\Owner2\\Desktop\\G팔로미_vuno")
os.getcwd()

'C:\\Users\\Owner2\\Desktop\\G팔로미_vuno'

In [2]:
X=np.load('data_X_with_callback.npy')
y=np.load('data_y_with_callback.npy')

In [3]:
print(X.shape)
print(y.shape)

(2145, 224, 224, 3)
(2145, 4)


In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

# 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(224,224,3)))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 220, 220, 8)       2312      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 110, 110, 8)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 96800)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                6195264   
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 260       
Total params: 6,198,732
Trainable params: 6,198,732
Non-trainable params: 0
____________________________________________

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
#train, validation set 분리
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2)

In [14]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True, 
    rotation_range=15.0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=[0.8,1.2],
    horizontal_flip=True) #수평방향 뒤집기

datagen.fit(X_train)

dgf=datagen.flow(X_train,y_train,batch_size=32)

In [15]:
history=model.fit_generator(dgf,
                   epochs=20,
                   steps_per_epoch=100,
                   validation_data=(X_val,y_val))

Epoch 1/20
100/100 [==============================] - 31s 308ms/step - loss: 2.9142 - accuracy: 0.3087 - val_loss: 1.3794 - val_accuracy: 0.3403
Epoch 2/20
100/100 [==============================] - 28s 284ms/step - loss: 1.3700 - accuracy: 0.3635 - val_loss: 1.3704 - val_accuracy: 0.3403
Epoch 3/20
100/100 [==============================] - 29s 287ms/step - loss: 1.3578 - accuracy: 0.3520 - val_loss: 1.3675 - val_accuracy: 0.3403
Epoch 4/20
100/100 [==============================] - 29s 292ms/step - loss: 1.3487 - accuracy: 0.3661 - val_loss: 1.3680 - val_accuracy: 0.3403
Epoch 5/20
 12/100 [==>...........................] - ETA: 21s - loss: 1.3505 - accuracy: 0.3387

KeyboardInterrupt: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(loss, 'y', label='train loss')
loss_ax.plot(val_loss, 'r', label='val loss')

acc_ax.plot(acc, 'b', label='train acc')
acc_ax.plot(val_acc, 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [35]:
X_test=np.load('X_test.npy')
y_test=np.load('y_test.npy')

In [36]:
# 성능 측정
loss,metrics = model.evaluate(X_test,y_test, batch_size=32)
print('## evaluation loss and_metrics ##')
print("loss :",loss)
print("accuracy :", metrics)

326/326 [==============================] - 1s 4ms/step
## evaluation loss and_metrics ##
loss : 10916.763289188315
accuracy : 0.401840478181839
